In [ ]:
from aiidalab_ispg.widgets import TrajectoryDataViewer, TrajectoryManagerWidget
from aiidalab_ispg import ConformerSmilesWidget
from aiidalab_ispg.conformers import FFMethod, XTBMethod, RDKitMethod
import ipywidgets as ipw

In [ ]:
rdkit_selector = ipw.Dropdown(
    options=[(method.value, method) for method in RDKitMethod],
    value=RDKitMethod.ETKDGV3,
)
ff_selector = ipw.Dropdown(
    options=[(method.value, method) for method in FFMethod],
    value=FFMethod.MMFF94,
)
xtb_selector = ipw.Dropdown(
    options=[(method.value, method) for method in XTBMethod],
    value=XTBMethod.NONE,
)
controls = ipw.HBox(children=[rdkit_selector, ff_selector, xtb_selector])

In [ ]:
conformer_widget = ConformerSmilesWidget(title="Conformer sampling")
conformer_widget.debug = True
ipw.dlink((ff_selector, 'value'), (conformer_widget, 'ff_method'))
ipw.dlink((rdkit_selector, 'value'), (conformer_widget, 'rdkit_method'))
ipw.dlink((xtb_selector, 'value'), (conformer_widget, 'xtb_method'))

structure_manager_widget = TrajectoryManagerWidget(
    importers=[conformer_widget],
    node_class='TrajectoryData',
    viewer=TrajectoryDataViewer(),
)
display(ipw.VBox(children=[controls, structure_manager_widget]))